In [1]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

class TransformerModel(nn.Module):

    def __init__(self, ntoken, ninp, nhead, nhid, nlayers, dropout=0.5):
        super(TransformerModel, self).__init__()
        from torch.nn import TransformerEncoder, TransformerEncoderLayer
        self.model_type = 'Transformer'
        self.src_mask = None
        self.pos_encoder = PositionalEncoding(ninp, dropout)
        encoder_layers = TransformerEncoderLayer(ninp, nhead, nhid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.encoder = nn.Embedding(ntoken, ninp)
        self.ninp = ninp
        self.decoder = nn.Linear(ninp, ntoken)

        self.init_weights()

    def _generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src):
        if self.src_mask is None or self.src_mask.size(0) != len(src):
            device = src.device
            mask = self._generate_square_subsequent_mask(len(src)).to(device)
            self.src_mask = mask

        src = self.encoder(src) * math.sqrt(self.ninp)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, self.src_mask)
        output = self.decoder(output)
        return output

In [50]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

In [4]:
import torchtext
from torchtext.data.utils import get_tokenizer
TEXT = torchtext.data.Field(tokenize=get_tokenizer("basic_english"),
                            init_token='<sos>',
                            eos_token='<eos>',
                            lower=True)
train_txt, val_txt, test_txt = torchtext.datasets.WikiText2.splits(TEXT)
TEXT.build_vocab(train_txt)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def batchify(data, bsz):
    data = TEXT.numericalize([data.examples[0].text])
    # Divide the dataset into bsz parts.
    nbatch = data.size(0) // bsz
    # Trim off any extra elements that wouldn't cleanly fit (remainders).
    data = data.narrow(0, 0, nbatch * bsz)
    # Evenly divide the data across the bsz batches.
    data = data.view(bsz, -1).t().contiguous()
    return data.to(device)

batch_size = 20
eval_batch_size = 10
train_data = batchify(train_txt, batch_size)
val_data = batchify(val_txt, eval_batch_size)
test_data = batchify(test_txt, eval_batch_size)

downloading wikitext-2-v1.zip


wikitext-2-v1.zip: 100%|██████████| 4.48M/4.48M [00:04<00:00, 1.00MB/s]


extracting


In [48]:
bptt = 35
def get_batch(source, i):
    seq_len = min(bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].view(-1)
    return data, target

In [51]:
ntokens = len(TEXT.vocab.stoi) # the size of vocabulary
emsize = 200 # embedding dimension
nhid = 200 # the dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 2 # the number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 2 # the number of heads in the multiheadattention models
dropout = 0.2 # the dropout value
model = TransformerModel(ntokens, emsize, nhead, nhid, nlayers, dropout).to(device)

In [59]:
criterion = nn.CrossEntropyLoss()
lr = 5.0 # learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

import time
import tqdm as tqdm
def train():
    model.train() # Turn on the train mode
    total_loss = 0.
    start_time = time.time()
    ntokens = len(TEXT.vocab.stoi)
    print("Batches:"+str((train_data.size(0) - 1)// bptt))
    for batch, i in tqdm.tqdm(enumerate(range(0, train_data.size(0) - 1, bptt))):
        data, targets = get_batch(train_data, i)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output.view(-1, ntokens), targets)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()
        log_interval = 200
        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | '
                  'lr {:02.2f} | ms/batch {:5.2f} | '
                  'loss {:5.2f} | ppl {:8.2f}'.format(
                    epoch, batch, len(train_data) // bptt, scheduler.get_lr()[0],
                    elapsed * 1000 / log_interval,
                    cur_loss, math.exp(cur_loss)))
            total_loss = 0
            start_time = time.time()

def evaluate(eval_model, data_source):
    eval_model.eval() # Turn on the evaluation mode
    total_loss = 0.
    ntokens = len(TEXT.vocab.stoi)
    with torch.no_grad():
        for i in range(0, data_source.size(0) - 1, bptt):
            data, targets = get_batch(data_source, i)
            output = eval_model(data)
            output_flat = output.view(-1, ntokens)
            total_loss += len(data) * criterion(output_flat, targets).item()
    return total_loss / (len(data_source) - 1)

In [62]:
best_val_loss = float("inf")
epochs = 10 # The number of epochs
best_model = None

for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train()
    val_loss = evaluate(model, val_data)
    print('-' * 89)
    print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f} | '
          'valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
                                     val_loss, math.exp(val_loss)))
    print('-' * 89)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = model

    scheduler.step()



0it [00:00, ?it/s]

1it [00:02,  2.91s/it]

2it [00:05,  2.83s/it]

3it [00:08,  2.79s/it]

4it [00:10,  2.53s/it]

5it [00:11,  2.22s/it]

6it [00:13,  2.04s/it]

7it [00:14,  1.93s/it]

8it [00:17,  2.14s/it]

9it [00:19,  2.20s/it]

10it [00:21,  2.13s/it]

11it [00:24,  2.26s/it]

12it [00:26,  2.12s/it]

13it [00:28,  2.08s/it]

14it [00:30,  2.19s/it]

15it [00:32,  1.97s/it]

16it [00:33,  1.92s/it]

17it [00:35,  1.93s/it]

18it [00:39,  2.29s/it]

19it [00:42,  2.50s/it]

20it [00:45,  2.64s/it]

21it [00:47,  2.74s/it]

22it [00:52,  3.13s/it]

23it [00:55,  3.26s/it]

24it [00:59,  3.32s/it]

25it [01:03,  3.54s/it]

26it [01:06,  3.56s/it]

27it [01:11,  3.87s/it]

28it [01:15,  3.99s/it]

29it [01:19,  3.98s/it]

30it [01:22,  3.79s/it]

31it [01:25,  3.49s/it]

32it [01:30,  3.75s/it]

33it [01:32,  3.38s/it]

34it [01:34,  3.04s/it]

35it [01:36,  2.69s/it]

36it [01:38,  2.32s/it]

37it [01:39,  2.16s/it]

38it [01:42,  2.28s/it]

39it [01:44,  2.35s/it]

40it [01:47,

| epoch   1 |   200/ 2981 batches | lr 5.00 | ms/batch 1551.78 | loss  5.91 | ppl   369.56




202it [05:11,  1.08it/s]

203it [05:12,  1.09it/s]

204it [05:13,  1.10it/s]

205it [05:14,  1.01it/s]

206it [05:15,  1.02it/s]

207it [05:16,  1.06s/it]

208it [05:17,  1.03s/it]

209it [05:18,  1.02it/s]

210it [05:19,  1.03it/s]

211it [05:20,  1.02it/s]

212it [05:21,  1.00s/it]

213it [05:22,  1.01s/it]

214it [05:23,  1.01s/it]

215it [05:24,  1.01s/it]

216it [05:25,  1.00s/it]

217it [05:26,  1.01it/s]

218it [05:27,  1.03it/s]

219it [05:28,  1.03it/s]

220it [05:29,  1.01it/s]

221it [05:30,  1.03it/s]

222it [05:31,  1.05it/s]

223it [05:31,  1.05it/s]

224it [05:32,  1.07it/s]

225it [05:33,  1.07it/s]

226it [05:34,  1.07it/s]

227it [05:35,  1.09it/s]

228it [05:36,  1.03it/s]

229it [05:38,  1.17s/it]

230it [05:40,  1.52s/it]

231it [05:42,  1.72s/it]

232it [05:44,  1.80s/it]

233it [05:46,  1.66s/it]

234it [05:47,  1.47s/it]

235it [05:48,  1.32s/it]

236it [05:49,  1.22s/it]

237it [05:50,  1.15s/it]

238it [05:51,  1.07s/it]

239it [05:52,  1.04s/it]

240it [05:

| epoch   1 |   400/ 2981 batches | lr 5.00 | ms/batch 1003.82 | loss  5.85 | ppl   348.37




402it [08:32,  1.07it/s]

403it [08:33,  1.06it/s]

404it [08:33,  1.06it/s]

405it [08:34,  1.06it/s]

406it [08:35,  1.04it/s]

407it [08:37,  1.00it/s]

408it [08:37,  1.00it/s]

409it [08:38,  1.01it/s]

410it [08:39,  1.00it/s]

411it [08:40,  1.01it/s]

412it [08:41,  1.01it/s]

413it [08:42,  1.03it/s]

414it [08:43,  1.05it/s]

415it [08:44,  1.05it/s]

416it [08:45,  1.06it/s]

417it [08:46,  1.05it/s]

418it [08:47,  1.04it/s]

419it [08:48,  1.04it/s]

420it [08:49,  1.04it/s]

421it [08:50,  1.05it/s]

422it [08:51,  1.07it/s]

423it [08:52,  1.08it/s]

424it [08:53,  1.07it/s]

425it [08:54,  1.05it/s]

426it [08:55,  1.05it/s]

427it [08:56,  1.06it/s]

428it [08:57,  1.05it/s]

429it [08:58,  1.01it/s]

430it [08:59,  1.04s/it]

431it [09:00,  1.16s/it]

432it [09:01,  1.12s/it]

433it [09:02,  1.06s/it]

434it [09:03,  1.02s/it]

435it [09:04,  1.02it/s]

436it [09:05,  1.02it/s]

437it [09:06,  1.03s/it]

438it [09:07,  1.05s/it]

439it [09:08,  1.03s/it]

440it [09:

...

| epoch   6 |  1800/ 2981 batches | lr 3.87 | ms/batch 972.94 | loss  5.01 | ppl   149.42




1802it [29:40,  1.06it/s]

1803it [29:41,  1.07it/s]

1804it [29:42,  1.07it/s]

1805it [29:43,  1.09it/s]

1806it [29:44,  1.09it/s]

1807it [29:45,  1.10it/s]

1808it [29:45,  1.12it/s]

1809it [29:46,  1.13it/s]

1810it [29:47,  1.13it/s]

1811it [29:48,  1.11it/s]

1812it [29:49,  1.12it/s]

1813it [29:50,  1.11it/s]

1814it [29:51,  1.11it/s]

1815it [29:52,  1.10it/s]

1816it [29:53,  1.11it/s]

1817it [29:54,  1.09it/s]

1818it [29:54,  1.10it/s]

1819it [29:55,  1.10it/s]

1820it [29:56,  1.09it/s]

1821it [29:57,  1.10it/s]

1822it [29:58,  1.09it/s]

1823it [29:59,  1.09it/s]

1824it [30:00,  1.08it/s]

1825it [30:01,  1.09it/s]

1826it [30:02,  1.09it/s]

1827it [30:03,  1.09it/s]

1828it [30:04,  1.07it/s]

1829it [30:05,  1.08it/s]

1830it [30:06,  1.07it/s]

1831it [30:07,  1.07it/s]

1832it [30:07,  1.08it/s]

1833it [30:08,  1.08it/s]

1834it [30:09,  1.08it/s]

1835it [30:10,  1.10it/s]

1836it [30:11,  1.09it/s]

1837it [30:12,  1.10it/s]

1838it [30:13,  1.09it/s]


| epoch   6 |  2000/ 2981 batches | lr 3.87 | ms/batch 964.47 | loss  5.06 | ppl   157.17




2002it [32:53,  1.07it/s]

2003it [32:54,  1.08it/s]

2004it [32:55,  1.07it/s]

2005it [32:56,  1.08it/s]

2006it [32:57,  1.09it/s]

2007it [32:58,  1.09it/s]

2008it [32:58,  1.09it/s]

2009it [32:59,  1.09it/s]

2010it [33:00,  1.09it/s]

2011it [33:01,  1.09it/s]

2012it [33:02,  1.09it/s]

2013it [33:03,  1.09it/s]

2014it [33:04,  1.08it/s]

2015it [33:05,  1.06it/s]

2016it [33:06,  1.07it/s]

2017it [33:07,  1.06it/s]

2018it [33:08,  1.01s/it]

2019it [33:09,  1.10s/it]

2020it [33:10,  1.08s/it]

2021it [33:11,  1.07s/it]

2022it [33:12,  1.03s/it]

2023it [33:13,  1.00s/it]

2024it [33:14,  1.02it/s]

2025it [33:15,  1.01it/s]

2026it [33:16,  1.01it/s]

2027it [33:17,  1.04it/s]

2028it [33:18,  1.06it/s]

2029it [33:19,  1.05it/s]

2030it [33:20,  1.05it/s]

2031it [33:21,  1.05it/s]

2032it [33:22,  1.06it/s]

2033it [33:23,  1.07it/s]

2034it [33:24,  1.07it/s]

2035it [33:25,  1.06it/s]

2036it [33:26,  1.03it/s]

2037it [33:27,  1.05it/s]

2038it [33:27,  1.06it/s]


KeyboardInterrupt: 

In [41]:
TEXT.vocab.stoi

defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x118a7c080>>,
            {'<unk>': 0,
             '<pad>': 1,
             '<sos>': 2,
             '<eos>': 3,
             'the': 4,
             ',': 5,
             '.': 6,
             'of': 7,
             'and': 8,
             'in': 9,
             'to': 10,
             'a': 11,
             '=': 12,
             'was': 13,
             "'": 14,
             '@-@': 15,
             'on': 16,
             'as': 17,
             's': 18,
             'that': 19,
             'for': 20,
             'with': 21,
             'by': 22,
             ')': 23,
             '(': 24,
             '@': 25,
             'is': 26,
             'it': 27,
             'from': 28,
             'at': 29,
             'his': 30,
             'he': 31,
             'were': 32,
             'an': 33,
             'had': 34,
             'which': 35,
             'be': 36,
             'are': 37,
            

In [10]:
train_data[0]

tensor([   3,   25, 1849,  570,    7,    5,    5, 9258,    4,   56,    0,    7,
           6, 6634,    4, 6603,    6,    5,   65,   30])